<a href="https://colab.research.google.com/github/richardenrico/Pengantar-Deep-Learning/blob/main/Week_3_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Week 3 - Deep Learning**

Richard Enrico Sulieanto

H071191055

In [1]:
import tensorflow as tf

# 5.1. Layers and Blocks

**MLP**

In [2]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.06286082,  0.02025608, -0.32948464, -0.07619414,  0.09632549,
        -0.01411874, -0.11833848, -0.23104535, -0.18344498, -0.10417721],
       [-0.11417283,  0.11048095, -0.45912415,  0.04704086,  0.10047662,
         0.05898417,  0.04477847, -0.3558252 , -0.26186574, -0.04591203]],
      dtype=float32)>

## 5.1.1. A Custom Block

In [3]:
class MLP(tf.keras.Model):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Model` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        # Hidden layer
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def call(self, X):
        return self.out(self.hidden((X)))

In [4]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.20925586,  0.20492466, -0.11957157, -0.09066397, -0.17926605,
        -0.00820188,  0.33358812,  0.17794555,  0.10594914, -0.01208188],
       [-0.06686239,  0.03489172, -0.05354899, -0.0234943 ,  0.01339397,
         0.10405202,  0.19524336,  0.17850715,  0.15307313,  0.07289076]],
      dtype=float32)>

## 5.1.2. The Sequential Block

In [5]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            # Here, `block` is an instance of a `tf.keras.layers.Layer`
            # subclass
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [6]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.28106686, -0.14132322,  0.24851654, -0.06834551, -0.4189058 ,
        -0.03097619, -0.1723272 ,  0.1702821 , -0.09144492, -0.00568734],
       [-0.2822415 , -0.33100575,  0.45612305, -0.14080371, -0.4667687 ,
         0.03599526, -0.2899044 ,  0.34233686, -0.2077746 , -0.19524921]],
      dtype=float32)>

## 5.1.3. Executing Code in the Forward Propagation Function

In [8]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        # Random weight parameters created with `tf.constant` are not updated
        # during training (i.e., constant parameters)
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        # Use the created constant parameters, as well as the `relu` and
        # `matmul` functions
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.dense(X)
        # Control flow
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In [9]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9027067>

In [10]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7344571>

# 5.2. Parameter Management

**MLP**

In [11]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.00549989],
       [ 0.16979554]], dtype=float32)>

## 5.2.1. Parameter Access

In [12]:
print(net.layers[2].weights)

[<tf.Variable 'dense_13/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.80027676],
       [-1.0367182 ],
       [ 0.29173768],
       [-0.9289979 ]], dtype=float32)>, <tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


### 5.2.1.1. Targeted Parameters

In [13]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


### 5.2.1.2. All Parameters at Once

In [14]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense_12/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.23622137, -0.38229153, -0.23112243,  0.15299517],
       [-0.45354432,  0.7459298 ,  0.55909115, -0.8061955 ],
       [ 0.7403694 ,  0.1462577 ,  0.09154612,  0.4112478 ],
       [ 0.3550629 ,  0.584783  ,  0.75858325,  0.02982944]],
      dtype=float32)>, <tf.Variable 'dense_12/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[ 0.23622137, -0.38229153, -0.23112243,  0.15299517],
       [-0.45354432,  0.7459298 ,  0.55909115, -0.8061955 ],
       [ 0.7403694 ,  0.1462577 ,  0.09154612,  0.4112478 ],
       [ 0.3550629 ,  0.584783  ,  0.75858325,  0.02982944]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[ 0.80027676],
       [-1.0367182 ],
       [ 0.29173768],
       [-0.9289979 ]], dtype=float32), array([0.], dtype=float32)]


In [15]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

### 5.2.1.3. Collecting Parameters from Nested Blocks

In [16]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.05970864],
       [-0.08194268]], dtype=float32)>

In [17]:
print(rgnet.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (2, 4)                    80        
                                                                 
 dense_18 (Dense)            (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_15/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

## 5.2.2. Parameter Initialization

### 5.2.2.1. Built-in Initialization

In [19]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_19/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.00439087, -0.00185346, -0.00403035, -0.00543838],
        [ 0.00025853, -0.00360808,  0.00543951, -0.01292864],
        [ 0.0286948 ,  0.00648367,  0.00243614, -0.00371349],
        [ 0.00649854, -0.00176175,  0.00767108,  0.00373384]],
       dtype=float32)>,
 <tf.Variable 'dense_19/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [20]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_21/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_21/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [22]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.35057652, -0.2303822 ,  0.7190147 , -0.7281951 ],
       [ 0.79380924, -0.5519184 ,  0.17863542, -0.65650356],
       [-0.8180198 , -0.39479512, -0.01217139, -0.65636206],
       [-0.5545334 ,  0.49744576, -0.77059007, -0.6978135 ]],
      dtype=float32)>
<tf.Variable 'dense_26/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>


### 5.2.2.2. Custom Initialization

In [23]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_27/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.       , -8.667204 ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       , -0.       ],
       [ 0.       , -0.       ,  9.283634 ,  5.7205276],
       [-0.       ,  9.986771 , -0.       , -7.6383424]], dtype=float32)>


In [24]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_27/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       , -7.667204 ,  1.       ,  1.       ],
       [ 1.       ,  1.       ,  1.       ,  1.       ],
       [ 1.       ,  1.       , 10.283634 ,  6.7205276],
       [ 1.       , 10.986771 ,  1.       , -6.6383424]], dtype=float32)>

### 5.2.3. Tied Parameters

In [25]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


# 5.3. Deferred Initialization

## 5.3.1. Instantiating a Network

In [26]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [27]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [28]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

# 5.4. Custom Layers

## 5.4.1. Layers without Parameters

In [29]:
class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [30]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [31]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [32]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=1.3969839e-09>

## 5.4.2. Layers with Parameters

In [33]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [34]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[ 0.02458033, -0.00901035,  0.03493255],
        [-0.03489552,  0.05251938, -0.01817408],
        [ 0.05945728, -0.00426352,  0.05153639],
        [ 0.01542093,  0.03508597, -0.03973434],
        [ 0.01979029, -0.02467043,  0.00958792]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [35]:
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.04403341, 0.02865572, 0.        ],
       [0.02890346, 0.04521246, 0.01922016]], dtype=float32)>

In [36]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.00463131],
       [0.0053801 ]], dtype=float32)>

# 5.5. File I/O



## 5.5.1. Loading and Saving Tensors

In [37]:
import numpy as np

x = tf.range(4)
np.save('x-file.npy', x)

In [38]:
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [39]:
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

In [40]:
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

## 5.5.2. Loading and Saving Model Parameters

In [41]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

In [42]:
net.save_weights('mlp.params')

In [43]:
clone = MLP()
clone.load_weights('mlp.params')

In [44]:
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

# 5.6. GPUs

In [46]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## 5.6.1. Computing Devices

In [47]:
tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f25338c2a00>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f25338be370>)

In [48]:
len(tf.config.experimental.list_physical_devices('GPU'))

0

In [50]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f2533964f50>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x7f2533964320>])

## 5.6.2. Tensors and GPUs

In [51]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

### 5.6.2.1. Storage on the GPU

In [52]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [53]:
with try_gpu(1):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.48404765, 0.6336924 , 0.03100729],
       [0.40050566, 0.11398005, 0.9822823 ]], dtype=float32)>

### 5.6.2.2. Copying

In [54]:
with try_gpu(1):
    Z = X
print(X)
print(Z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [55]:
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.4840477, 1.6336924, 1.0310073],
       [1.4005057, 1.11398  , 1.9822823]], dtype=float32)>

In [56]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True

## 5.6.3. Neural Networks and GPUs

In [57]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [58]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[1.1335244],
       [1.1335244]], dtype=float32)>

In [59]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:CPU:0',
 '/job:localhost/replica:0/task:0/device:CPU:0')